<p style="text-align: center"> Escuela Técnica Superior de Ingeniería Informática</p>
<p style="text-align: center">Universidad de Sevilla</p>
<p style="text-align: center">Síntesis, Verificación y Razonamiento sobre Agentes Inteligentes </p>
<p style="text-align: center"> EJERCICIOS </p>
<p> </p>

**Nombre del alumno:**
- Kenny Jesús Flores Huamán

<!-- 
Para generar la tabla de contenidos que se va a haber a continuación, se ha hecho uso del siguiente código
URL : https://stackoverflow.com/questions/21151450/how-can-i-add-a-table-of-contents-to-a-jupyter-jupyterlab-notebook#:~:text=Click%20the%20toc2%20symbol%20in,you%20open%20it%20in%20JupyterLab.

import json
import urllib
def generate_toc(notebook_path, indent_char="&emsp;"):
    is_markdown = lambda it: "markdown" == it["cell_type"]
    is_title = lambda it: it.strip().startswith("#") and it.strip().lstrip("#").lstrip()
    with open(notebook_path, 'r') as in_f:
        nb_json = json.load(in_f)
    for cell in filter(is_markdown, nb_json["cells"]):
        for line in filter(is_title, cell["source"]):
            line = line.strip()
            indent = indent_char * (line.index(" ") - 1)
            title = line.lstrip("#").lstrip()
            url = urllib.parse.quote(title.replace(" ", "-"))
            out_line = f"{indent}[{title}](#{url})<br>\n"
            print(out_line, end="")
            
generate_toc('C6_FloresHuaman.ipynb')


-->

# Introduccion

En este notebook se van a realizar la resolución de los diferentes ejercicios mostrados en la página oficial de la asignatura.
Todos los archivos que se van a importar que no sean librerías estándar de Python se encuentran en el mismo directorio de donde se encuentra este notebook.

In [6]:
import warnings
warnings.filterwarnings('ignore')

# VEHICULO EN PENDIENTE

## Versión 1

Consideremos un vehículo que opera en una pendiente y utiliza paneles solares para recargarse. Puede encontrarse en uno de los tres estados de la pendiente: alto, medio y bajo. Si hace girar sus ruedas, sube la pendiente en cada paso de tiempo (de bajo a medio o de medio a alto) o se mantiene alto. Si no gira las ruedas, desciende por la pendiente en cada paso de tiempo (de alto a medio o de medio a bajo) o se mantiene bajo. Hacer girar las ruedas consume una unidad de energía por paso de tiempo. Estando en la parte alta o media de la pendiente gana tres unidades de energía por paso de tiempo a través de los paneles solares, mientras que estando en la parte baja de la pendiente no gana nada de energía por paso de tiempo. El robot quiere ganar tanta energía como sea posible

1. Representa gráficamente el MDP.
2. Resuelve el MDP utilizando la iteración de valores con un factor de descuento de 0,8.
3. Describe la política óptima.


#### Importacion de las librerias 

In [17]:
from vehiclesplopeV1 import *

In [ ]:
vehicle = VehicleSlopeV1(discount_factor=0.8)

## Versión 2 

Supongamos ahora que el vehículo puede encontrarse en uno de los cuatro estados de la pendiente: superior, alto, medio y bajo. Si gira sus ruedas lentamente, sube la pendiente en cada paso de tiempo (de bajo a medio, o de medio a alto, o de alto a superior) con una probabilidad de 0,3
, y desciende por la pendiente hasta la parte baja con una probabilidad de 0,7
. Si hace girar sus ruedas rápidamente, sube la pendiente en cada paso de tiempo (de bajo a medio, o de medio a alto, o de alto a superior) con una probabilidad de 0,7
, y se desliza por la pendiente hasta llegar abajo con una probabilidad de 0,3
. El giro lento de las ruedas consume una unidad de energía por paso de tiempo. Girar las ruedas rápidamente consume dos unidades de energía por unidad de tiempo. El vehículo se encuentra en la parte baja de la pendiente y su objetivo es llegar a la cima con el mínimo consumo de energía previsto.

1. Representa gráficamente el MDP.
2. Resuelve el MDP utilizando iteración de valor sin descuento (es decir, iteración de valor con un factor de descuento de 1).
3. Describe la política óptima.


### Importacion de las librerias 

In [1]:
from vehiclesplopeV2 import *

### Instanciación del problema

In [6]:
# Vamos a instanciando con un valor del factor de descuento de 1 
vehicle2 = VehicleSlopeV2(discount_factor=1)

In [7]:
values = TabularValueFunction()
ValueIteration(vehicle2 values).value_iteration(max_iterations=100)
policy = values.extract_policy(vehicle)

In [8]:
policy.print_policy_table()

+----------+---------------+
|  Estado  |     Acción    |
+----------+---------------+
| LOW      | SPIN_FAST     |
+----------+---------------+
| MEDIUM   | SPIN_FAST     |
+----------+---------------+
| HIGH     | SPIN_FAST     |
+----------+---------------+
| TOP      | SPIN_LOW      |
+----------+---------------+


# MUNDO MALLA 

El ejemplo de mundo malla es una idealización del movimiento de un robot en un entorno. En cada momento, el robot se encuentra en una ubicación y puede desplazarse a las ubicaciones vecinas, recogiendo recompensas y castigos. Supongamos que las acciones son estocásticas, de modo que existe una distribución de probabilidad sobre los estados resultantes dada la acción y el estado.

![Descripción de la imagen](grid.png)


En la figura adjunta se muestra un mundo cuadriculado de 10×10
, en el que el robot puede elegir una de cuatro acciones: arriba, abajo, izquierda o derecha. Si el agente lleva a cabo una de estas acciones, tiene una probabilidad de 0.7
 de dar un paso en la dirección deseada y una probabilidad de 0.1
 de dar un paso en cualquiera de las otras tres direcciones. Si choca contra la pared exterior (es decir, la ubicación calculada está fuera de la malla), tiene una penalización de 1
 (es decir, una recompensa de −1
) y el agente no se mueve. Hay cuatro estados con recompensa (aparte de las paredes): +10
 (en la posición (9,8)
), +3
 (en (8,3)
), −5
 (en (4,5)
) y −10
 (en (4,8)
). En cada uno de estos estados, el agente obtiene la recompensa después de realizar una acción en ese estado, no cuando entra en él. Cuando el agente alcanza uno de los estados con recompensa positiva (ya sea +3
 o +10
), independientemente de la acción que realice, en el siguiente paso es lanzado, al azar, a una de las cuatro esquinas del mundo cuadriculado.

1. Representa gráficamente el MDP.
2. Resuelve el MDP por diversos métodos.
3. Describe la política óptima.

## Representación del MDP

### Importacion de librerias

In [1]:
from gridworld import *
from policy_iteration import PolicyIteration
from tabular_policy import TabularPolicy
from qtable import QTable
from qlearning import QLearning
from sarsa import SARSA
from multi_armed_bandit import EpsilonGreedy
from tabular_value_function import TabularValueFunction
from value_iteration import ValueIteration

pygame 2.1.2 (SDL 2.0.16, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Instancia del mundo malla 10x10

In [2]:
gridworld = GridWorld(goals=[((8, 2), +10), ((7, 7), +3),
                     ((3, 5), -5), ((3, 2), -10)])

### Representacion del estado inicial de la malla

In [4]:
gridworld.visualise_initial_state()

SystemExit: 

/home/kenny/Documentos/PyCA/env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Resolucion del MDP por diversos métodos

### Resolucion del MDP mediante Iteración de valores

In [3]:
values = TabularValueFunction()
ValueIteration(gridworld, values).value_iteration(max_iterations=100)
policy = values.extract_policy(gridworld)
print(gridworld.policy_to_string(policy))


  |  →  |  →  |  →  |  →  |  →  |  →  |  →  |  ↓  |  ↓  |  ↓ 
 |  →  |  →  |  →  |  →  |  →  |  →  |  ↓  |  ↓  |  ↓  |  ↓ 
 |  →  |  →  |  →  |  →  |  →  |  →  |  →  | 3  |  ↓  |  ↓ 
 |  →  |  →  |  →  |  ↑  |  →  |  →  |  →  |  →  |  ↓  |  ↓ 
 |  ↓  |  →  |  ←  | -5  |  →  |  →  |  →  |  ↓  |  ↓  |  ↓ 
 |  ↓  |  →  |  →  |  ↓  |  →  |  →  |  →  |  ↓  |  ↓  |  ↓ 
 |  ↓  |  →  |  →  |  ↑  |  →  |  →  |  →  |  →  |  ↓  |  ↓ 
 |  ↓  |  →  |  ←  | -10  |  →  |  →  |  →  |  →  | 10  |  ← 
 |  ↓  |  →  |  →  |  ↓  |  →  |  →  |  →  |  →  |  ↑  |  ↑ 
 |  →  |  →  |  →  |  →  |  →  |  →  |  →  |  →  |  ↑  |  ↑ 



In [4]:
policy.print_policy_table(15,10)

+------------------------------+
| Estado          | Accion     |
+------------------------------+
| ('end', 'end')  | end        |
+------------------------------+
| (0, 0)          | RIGHT      |
+------------------------------+
| (0, 1)          | DOWN       |
+------------------------------+
| (0, 2)          | DOWN       |
+------------------------------+
| (0, 3)          | DOWN       |
+------------------------------+
| (0, 4)          | DOWN       |
+------------------------------+
| (0, 5)          | DOWN       |
+------------------------------+
| (0, 6)          | RIGHT      |
+------------------------------+
| (0, 7)          | RIGHT      |
+------------------------------+
| (0, 8)          | RIGHT      |
+------------------------------+
| (0, 9)          | RIGHT      |
+------------------------------+
| (1, 0)          | RIGHT      |
+------------------------------+
| (1, 1)          | RIGHT      |
+------------------------------+
| (1, 2)          | RIGHT      |
+---------

### Resolucion del MDP mediante Iteración de políticas

In [8]:
policy = TabularPolicy(default_action=gridworld.UP)
PolicyIteration(gridworld, policy).policy_iteration(max_iterations=300)
print(gridworld.policy_to_string(policy))


  |  →  |  →  |  →  |  →  |  →  |  →  |  →  |  ↓  |  ↓  |  ↓ 
 |  →  |  →  |  →  |  →  |  →  |  →  |  ↓  |  ↓  |  ↓  |  ↓ 
 |  →  |  →  |  →  |  →  |  →  |  →  |  →  | 3  |  ↓  |  ↓ 
 |  →  |  →  |  →  |  ↑  |  →  |  →  |  →  |  →  |  ↓  |  ↓ 
 |  ↓  |  →  |  ←  | -5  |  →  |  →  |  →  |  ↓  |  ↓  |  ↓ 
 |  ↓  |  →  |  →  |  ↓  |  →  |  →  |  →  |  ↓  |  ↓  |  ↓ 
 |  ↓  |  →  |  →  |  ↑  |  →  |  →  |  →  |  →  |  ↓  |  ↓ 
 |  ↓  |  →  |  ←  | -10  |  →  |  →  |  →  |  →  | 10  |  ← 
 |  ↓  |  →  |  →  |  ↓  |  →  |  →  |  →  |  →  |  ↑  |  ↑ 
 |  →  |  →  |  →  |  →  |  →  |  →  |  →  |  →  |  ↑  |  ↑ 



### Resolucion del MDP mediante Q-Learning

In [6]:
qfunction = QTable()
QLearning(gridworld, EpsilonGreedy(), qfunction).execute(episodes=2000)
policy = qfunction.extract_policy(gridworld)

Episodes: 100%|████████████████████████████| 2000/2000 [00:06<00:00, 316.10it/s]


In [7]:
print(gridworld.policy_to_string(policy))  


  |  ↓  |  →  |  →  |  →  |  →  |  →  |  →  |  ↓  |  ←  |  ↑ 
 |  →  |  →  |  →  |  →  |  →  |  →  |  ↓  |  ←  |  ←  |  ↑ 
 |  →  |  →  |  →  |  ↑  |  →  |  →  |  →  | 3  |  ←  |  ↑ 
 |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑ 
 |  ↑  |  ↑  |  ←  | -5  |  →  |  ↑  |  ↑  |  ←  |  ↑  |  ↑ 
 |  →  |  ↑  |  ↑  |  ↓  |  ←  |  →  |  →  |  ↑  |  ↑  |  ↑ 
 |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ←  |  ↓  |  ↑  |  ← 
 |  ↑  |  ↑  |  ←  | -10  |  →  |  ↓  |  →  |  ↓  | 10  |  ↑ 
 |  ↑  |  ↑  |  ↑  |  ↓  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑ 
 |  ↑  |  ←  |  ↑  |  ←  |  ↑  |  ←  |  ↑  |  ↑  |  ↑  |  ↑ 



### Resolucion del MDP mediante Sarsa

In [14]:
qfunction = QTable()
SARSA(gridworld, EpsilonGreedy(), qfunction).execute(episodes=3000)
policy = qfunction.extract_policy(gridworld)

Episodes: 100%|████████████████████████████| 3000/3000 [00:04<00:00, 644.59it/s]


In [15]:
print(gridworld.policy_to_string(policy))  


  |  ↑  |  →  |  →  |  →  |  →  |  →  |  →  |  ←  |  ←  |  ↑ 
 |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  →  |  ↓  |  ↓  |  ←  |  ↑ 
 |  ↑  |  →  |  ↑  |  ↑  |  ↑  |  ↑  |  →  | 3  |  ↑  |  ↑ 
 |  ↑  |  ←  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑ 
 |  ←  |  ←  |  ←  | -5  |  →  |  ↑  |  ↑  |  ↑  |  ↑  |  ↑ 
 |  ←  |  ←  |  ←  |  ↓  |  ↑  |  →  |  ↑  |  ←  |  ↓  |  ↓ 
 |  →  |  ←  |  ↓  |  ↑  |  ↓  |  →  |  ↓  |  →  |  →  |  ↓ 
 |  ↓  |  ←  |  ←  | -10  |  →  |  →  |  →  |  →  | 10  |  ← 
 |  ↓  |  ↓  |  ↓  |  ↓  |  →  |  →  |  →  |  →  |  ↑  |  ↑ 
 |  →  |  →  |  →  |  →  |  →  |  →  |  →  |  →  |  ↑  |  ↑ 



## Descripción de la política óptima

# CartPole

Este problema, también llamado a veces problema de equilibrio de pértiga, permite que el agente mueva un carro horizontalmente. Como se muestra en la figura, el carro tiene una pértiga rígida unida a él por una rótula, de modo que cuando el carro se mueve hacia la derecha o izquierda, la pértiga puede girar alrededor de la unión. El objetivo es mantener el poste verticalmente equilibrado mientras se mantiene el carro dentro de los límites laterales permitidos. Se obtiene 1
 de recompensa cada unidad de tiempo en el que la pértiga no supera cierto ángulo de inclinación y el carro se encuentra dentro de ciertos límites laterales, y la transición a un estado terminal de recompensa 0
 se produce en caso contrario.

Las acciones consisten en aplicar una fuerza F
 a la izquierda o a la derecha del carro. El espacio de estados está definido por cuatro variables continuas: la posición lateral del carro x
, su velocidad lateral v
, el ángulo de la pértiga θ
 y la velocidad angular de la pértiga w
. Además, el problema involucra una variedad de parámetros necesarios para el modelado físico, que incluyen: la masa del carro mcart
, la masa de la pértiga mpole
, la longitud de la pértiga l
, la magnitud de la fuerza |F|
, la aceleración gravitacional g
, el paso de tiempo Δt
, la desviación x
 máxima, la desviación angular máxima, y las pérdidas por fricción entre el carro y la pértiga o entre el carro y la pista.

In [8]:
from cartpole import *

# Mountain Car

En el problema del coche de montaña un vehículo debe conducir hacia la derecha para salir de un valle. Las paredes del valle son lo suficientemente empinadas como para que acelerar a ciegas hacia la meta con una velocidad insuficiente haga que el vehículo se detenga y se deslice hacia abajo. El agente debe aprender a acelerar primero hacia la izquierda para ganar suficiente impulso en el retorno y poder subir la colina.

El estado es la posición horizontal del vehículo x∈[−1.2,0.6]
 y la velocidad v∈[−0.07,0.07]
. En cualquier paso de tiempo, el vehículo puede acelerar a la izquierda (a=−1
), acelerar a la derecha (a=1
), o dejarse llevar por la inercia (a=0
). Recibimos una recompensa de −1
 en cada cambio de sentido, y terminamos cuando el vehículo llega al lado derecho del valle más allá de x=0.6
.

La figura adjunta muestra una visualización animada de una ejecución del problema.

Las transiciones en el problema del coche de montaña son deterministas:

v′x′←←v+0.001 a−0.0025cos(3x)x+v′
El término gravitacional en la actualización de la velocidad es lo que impulsa al vehículo con poca potencia hacia el fondo del valle. Las transiciones se sujetan a los límites del espacio de estados.

El problema del coche de montaña es un buen ejemplo de un problema con retorno retardado. Se requieren muchas acciones para llegar al estado objetivo, lo que dificulta que un agente no entrenado reciba algo más que penalizaciones unitarias de forma continua. Los mejores algoritmos de aprendizaje son capaces de propagar eficazmente el conocimiento de las trayectorias que llegan a la meta al resto del espacio de estados.

In [16]:
from mountaincar import *